<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will build a model on customer churn binary classification problem. You will be using files_for_lab/Customer-Churn.csv file.

Scenario

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

Import the required libraries and modules that you would need.
Read that data into Python and call the dataframe churnData.
Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.
Check for null values in the dataframe. Replace the null values.
Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
Scale the features either by using normalizer or a standard scaler.
Split the data into a training set and a test set.
Fit a logistic regression model on the training data.
Check the accuracy on the test data.
Note: So far we have not balanced the data.

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

In [36]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [8]:
churnData = pd.read_csv('Customer-Churn.csv')

In [9]:
churnData.shape

(7043, 16)

In [10]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [11]:
#Checking the data types

churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [15]:
#Checking TotalCharges column

churnData['TotalCharges'].unique

<bound method Series.unique of 0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object>

In [16]:
#Converting TotalCharges to numeric

churnData[["TotalCharges"]] = churnData[["TotalCharges"]].apply(pd.to_numeric, errors='coerce')

In [18]:
#Checking for null values

nulls = pd.DataFrame(churnData.isna().sum()*100/len(data), columns=['percentage'])
nulls.sort_values('percentage', ascending = False)

,percentage
TotalCharges,0.156183
gender,0.000000
SeniorCitizen,0.000000
Partner,0.000000
Dependents,0.000000
tenure,0.000000
PhoneService,0.000000
OnlineSecurity,0.000000
OnlineBackup,0.000000
DeviceProtection,0.000000


In [19]:
#Only TotalCharges has null values. I will replace it with the mean

churnData[['TotalCharges']]=churnData[['TotalCharges']].fillna(value=churnData[['TotalCharges']].mean())

In [21]:
churnData['TotalCharges'].isna().sum()

0

Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
 Scale the features either by using normalizer or a standard scaler.
 Split the data into a training set and a test set.
 Fit a logistic regression model on the training data.
 Check the accuracy on the test data.
Note: So far we have not balanced the data.

In [25]:
data = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'Churn']]

In [26]:
data

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,1,0,29.85,29.85,No
1,34,0,56.95,1889.50,No
2,2,0,53.85,108.15,Yes
3,45,0,42.30,1840.75,No
4,2,0,70.70,151.65,Yes
...,...,...,...,...,...
7038,24,0,84.80,1990.50,No
7039,72,0,103.20,7362.90,No
7040,11,0,29.60,346.45,No
7041,4,1,74.40,306.60,Yes


In [27]:
#Splitting the data into train and test set. 

X = data.drop('Churn', axis=1)
y = data['Churn']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [35]:
#Scaling the features with StandardScaler

transformer = StandardScaler()
transformer.fit(X_train)

# saving in a pickle
with open('std_transformer.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer.pickle', 'rb') as file:
    loaded_transformer = pickle.load(file)

X_train_ = loaded_transformer.transform(X_train)
X_test_ = loaded_transformer.transform(X_test)

In [37]:
#Fitting the model using Logistic regression


classification = LogisticRegression(random_state=42, max_iter=1000)
classification.fit(X_train, y_train)
score = classification.score(X_test, y_test)
print (score)

0.8105039034776437


Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

In [38]:
#Checking for data imbalance. Data is imbalanced and Yes is the minority class

data['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [43]:
#Downsampling

category_no = data[data['Churn'] == 'No']
category_yes= data[data['Churn'] == 'Yes']

In [44]:
len(category_yes)

1869

In [45]:
category_no = category_no.sample(len(category_yes))
print(category_no.shape)
print(category_yes.shape)

(1869, 5)
(1869, 5)


In [47]:
data1 = pd.concat([category_no, category_yes], axis=0)
#shuffling the data
data1 = data.sample(frac=1)
data1['Churn'].value_counts()

Yes    1869
No     1869
Name: Churn, dtype: int64

In [48]:
X_train_down = data1.drop(['Churn'], axis=1)
y_train_down = data1['Churn']

In [49]:
classification = LogisticRegression(random_state=42, max_iter=1000)
classification.fit(X_train_down, y_train_down)
score = classification.score(X_test, y_test)
print (score)

0.7310149041873669
